In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import dhs_util
from dhs_util import *

os.chdir('C:/Users/lukeh/OneDrive/Documents/College/Topics in Data Science/Unit 1')
df = pd.read_csv('dhs_service_records_synthesized_final.csv')

df = dhs_preprocessing(df)
df, service_map = add_service_label(df)
df = add_age_bin(df)

recipient = get_recipient_attribute(df)

In [2]:
pip install mlxtend


Note: you may need to restart the kernel to use updated packages.


In [3]:
serv_list = []
for groups in df.groupby('id').groups.values():
    serv_list.append(df.loc[groups]['serv'].tolist())

In [4]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.preprocessing import *
from mlxtend.frequent_patterns import association_rules
from mlxtend.frequent_patterns import fpgrowth
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import fpmax
from mlxtend.frequent_patterns import hmine

In [8]:
# re-do the prep of list of services again
serv_list = []
for groups in df.groupby('id').groups.values():
    serv_list.append(df.loc[groups]['serv'].tolist())

# following the tutorial example 
def oneHotCoding(serv_list):
    te = TransactionEncoder()
    te_ary = te.fit(serv_list).transform(serv_list)
    te_df = pd.DataFrame(te_ary, columns=te.columns_)
    return te_df

serv_oneHot = oneHotCoding(serv_list)

In [10]:
min_freq = 1000 # if we want to set threshold by frequency of the itemsets
min_support = min_freq/serv_oneHot.shape[0]
min_confidence = 0.6
min_rule_support = 0.2
min_lift = 0.15

In [12]:
apriori(serv_oneHot, min_support=0.010) 

,support,itemsets
0,0.013687,(2)
1,0.012649,(4)
2,0.153844,(8)
3,0.040882,(10)
4,0.941422,(11)
5,0.011915,(12)
6,0.103528,(13)
7,0.024002,(14)
8,0.012967,(16)
9,0.013573,(17)


In [16]:
freq_itemset_fpgrowth = fpgrowth(serv_oneHot,min_support=min_support,use_colnames=True)
freq_itemset_fpgrowth.describe()

,support
count,146.000000
mean,0.017443
std,0.080320
min,0.001875
25%,0.002868
50%,0.004436
75%,0.009374
max,0.941422


In [14]:
hmine(serv_oneHot,min_support=0.0001,use_colnames=True)


,support,itemsets
0,0.000654,(S01)
1,0.000305,"(S12, S01)"
2,0.004191,(S02)
3,0.004142,"(S02, S03)"
4,0.000146,"(S02, S03, S04)"
...,...,...
930,0.000141,"(S21, S19)"
931,0.002471,(S20)
932,0.000126,"(S20, S21)"
933,0.01326,(S21)


Observations: 
Apriori produced 31 itemsets, while FPGrowth produced 146 itemsets, and HMine produced 64 itemsets. Apriori seems to return fewer itemsets because of its iterative nature and potentially stricter minimum support threshold (0.01). 

In [18]:
fpmax(serv_oneHot,min_support=0.0001,use_colnames=True)


,support,itemsets
0,0.000199,"(S12, S08)"
1,0.000305,"(S12, S01)"
2,0.000124,"(S22, S12)"
3,0.000133,"(S02, S03, S12, S04)"
4,0.000131,"(S14, S12, S04, S09, S03)"
...,...,...
59,0.000122,"(S14, S11, S12, S09, S21, S18)"
60,0.000210,"(S14, S12, S09, S21, S18, S15)"
61,0.000204,"(S14, S12, S09, S19, S18)"
62,0.000169,"(S14, S12, S09, S19, S15)"


Observations: 
FPMax produces fewer itemsets compared to FPGrowth or HMine because it returns only maximal itemsets. The main difference between FPMax and the other algorithms is that FPMax does not return all frequent itemsets, only those that are maximal.

In [22]:
freq_itemset_apriori = apriori(serv_oneHot, min_support=0.01, use_colnames=True)
length_1_itemsets_apriori = freq_itemset_apriori[freq_itemset_apriori['itemsets'].apply(lambda x: len(x) == 1)]
print(length_1_itemsets_apriori)


     support itemsets
0   0.013687    (S03)
1   0.012649    (S05)
2   0.153844    (S09)
3   0.040882    (S11)
4   0.941422    (S12)
5   0.011915    (S13)
6   0.103528    (S14)
7   0.024002    (S15)
8   0.012967    (S17)
9   0.013573    (S18)
10  0.031396    (S19)
11  0.013260    (S21)


In [24]:
freq_itemset_hmine = hmine(serv_oneHot, min_support=0.005, use_colnames=True)
length_2_itemsets_hmine = freq_itemset_hmine[freq_itemset_hmine['itemsets'].apply(lambda x: len(x) == 2)]
print(length_2_itemsets_hmine)


     support    itemsets
1   0.005274  (S03, S09)
3   0.012284  (S03, S12)
5   0.008057  (S05, S12)
8   0.006398  (S11, S07)
10  0.005105  (S12, S07)
12  0.018307  (S11, S09)
16  0.139131  (S12, S09)
27  0.008419  (S13, S09)
28  0.103528  (S14, S09)
32  0.019431  (S09, S15)
33  0.009372  (S16, S09)
34  0.006731  (S17, S09)
35  0.013468  (S18, S09)
36  0.005268  (S09, S19)
37  0.007949  (S21, S09)
39  0.005238  (S12, S10)
41  0.032106  (S11, S12)
43  0.009374  (S14, S11)
45  0.011448  (S13, S12)
46  0.094436  (S14, S12)
50  0.022561  (S12, S15)
51   0.00683  (S16, S12)
52  0.008674  (S17, S12)
53  0.010384  (S18, S12)
54  0.009949  (S12, S19)
55  0.008848  (S21, S12)
58  0.009888  (S14, S15)
59  0.009371  (S14, S16)
60   0.01346  (S14, S18)


In [26]:
freq_itemset_fpmax = fpmax(serv_oneHot, min_support=0.0001, use_colnames=True)
length_5_itemsets_fpmax = freq_itemset_fpmax[freq_itemset_fpmax['itemsets'].apply(lambda x: len(x) == 5)]
print(length_5_itemsets_fpmax)


     support                   itemsets
4   0.000131  (S14, S12, S04, S09, S03)
18  0.000163  (S14, S12, S10, S09, S18)
19  0.000157  (S12, S06, S10, S09, S21)
21  0.000124  (S12, S10, S09, S21, S15)
22  0.000208  (S14, S12, S10, S09, S15)
23  0.000139  (S14, S12, S10, S09, S21)
24  0.000197  (S14, S11, S12, S10, S09)
25  0.000103  (S14, S06, S09, S21, S16)
31  0.000103  (S11, S12, S09, S19, S07)
33  0.000120  (S11, S12, S09, S07, S03)
37  0.000131  (S14, S09, S19, S18, S16)
38  0.000142  (S14, S12, S09, S19, S16)
45  0.000120  (S14, S12, S09, S13, S03)
46  0.000111  (S14, S12, S17, S09, S13)
47  0.000150  (S14, S12, S09, S13, S15)
49  0.000150  (S14, S11, S12, S09, S13)
50  0.000111  (S14, S12, S09, S18, S05)
53  0.000109  (S11, S12, S17, S09, S21)
61  0.000204  (S14, S12, S09, S19, S18)
62  0.000169  (S14, S12, S09, S19, S15)
63  0.000247  (S14, S11, S12, S09, S19)


In [28]:
freq_itemset_fpgrowth = fpgrowth(serv_oneHot, min_support=0.05, use_colnames=True)
rules_fpgrowth_lift = association_rules(freq_itemset_fpgrowth, metric="lift", min_threshold=0.5)
print(rules_fpgrowth_lift[['antecedents', 'consequents', 'support', 'confidence', 'lift']])


   antecedents consequents   support  confidence      lift
0        (S12)       (S09)  0.139131    0.147788  0.960634
1        (S09)       (S12)  0.139131    0.904362  0.960634
2        (S14)       (S09)  0.103528    1.000000  6.500073
3        (S09)       (S14)  0.103528    0.672938  6.500073
4        (S14)       (S12)  0.094436    0.912184  0.968942
5        (S12)       (S14)  0.094436    0.100312  0.968942
6   (S14, S12)       (S09)  0.094436    1.000000  6.500073
7   (S14, S09)       (S12)  0.094436    0.912184  0.968942
8   (S12, S09)       (S14)  0.094436    0.678758  6.556292
9        (S14)  (S12, S09)  0.094436    0.912184  6.556292
10       (S12)  (S14, S09)  0.094436    0.100312  0.968942
11       (S09)  (S14, S12)  0.094436    0.613843  6.500073


In [36]:
freq_itemset_fpmax = fpmax(serv_oneHot, min_support=0.0001, use_colnames=True)
rules_fpmax_support = association_rules(freq_itemset_fpmax, metric="support", min_threshold=0.0003, support_only=True)
print(rules_fpmax_support[['antecedents', 'consequents', 'support']])


                  antecedents                consequents   support
0                       (S12)                      (S01)  0.000305
1                       (S01)                      (S12)  0.000305
2   (S14, S12, S17, S09, S18)                      (S15)  0.000326
3   (S14, S12, S17, S09, S15)                      (S18)  0.000326
4   (S14, S12, S17, S18, S15)                      (S09)  0.000326
..                        ...                        ...       ...
59                      (S12)  (S14, S17, S09, S18, S15)  0.000326
60                      (S17)  (S14, S12, S09, S18, S15)  0.000326
61                      (S09)  (S14, S12, S17, S18, S15)  0.000326
62                      (S18)  (S14, S12, S17, S09, S15)  0.000326
63                      (S15)  (S14, S12, S17, S09, S18)  0.000326

[64 rows x 3 columns]


In [52]:
def serv_rules(freq_itemsets,metrics,threshold):
    asso_rules = association_rules(freq_itemsets, metric=metrics, min_threshold=threshold)
    return asso_rules.sort_values(by='lift', ascending=False)[['antecedents', 'consequents', 'support', 'confidence', 'lift']]

def predict(antecedent, rules, consequents_only = False):
    preds = rules[rules['antecedents'] == antecedent] # get the rules for this antecedent
    if consequents_only:
        preds = preds['consequents'].apply(iter).apply(next) # a way to convert a frozen set with one element to string
    return preds

In [54]:
freq_itemset_apriori = apriori(serv_oneHot, use_colnames=True, min_support=min_support)\
    .sort_values(by="support", ascending=False)
freq_itemset_fpgrowth = fpgrowth(serv_oneHot,min_support=min_support,use_colnames=True)
freq_itemset_fpmax = fpmax(serv_oneHot,min_support=min_support,use_colnames=True)

rules_apriori = serv_rules(freq_itemset_apriori,"confidence",0.60)
rules_fpgrowth = serv_rules(freq_itemset_fpgrowth,"confidence",0.60)
rules_fpmax = association_rules(freq_itemset_fpmax, metric="confidence", min_threshold=0.001, support_only=True)

In [56]:
predict({"S06"}, rules_apriori, consequents_only=False).head()


,antecedents,consequents,support,confidence,lift
86,(S06),(S21),0.004582,0.784477,59.162639


In [60]:
predict({"S06"}, rules_fpgrowth, consequents_only=False).head()

,antecedents,consequents,support,confidence,lift
115,(S06),(S21),0.004582,0.784477,59.162639


In [62]:
predict({"S06"}, rules_fpmax, consequents_only=False).head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
37,(S06),"(S21, S12, S09)",NaN,NaN,0.002486,NaN,NaN,NaN,NaN,NaN


Conclusions: 
S21 is a critical service that should be considered for offering alongside S06 due to the strong associations shown in both the apriori and fpgrowth results. Based on the fpmax results, DHS should also consider offering a combination of S21, S12, and S09 together with S06, as these services seem to frequently co-occur, and may complement the needs of families receiving child welfare services.